# Modification du vocabulaire entre kalabas

In [1]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [2]:
from string import translate, maketrans

In [3]:
home = expanduser("~")
annee="23"
def getSerie(nomKalaba):
#     return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/%s-%s/"%(annee,nomKalaba)
    return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s-%s/"%(annee,nomKalaba)

prevSerie=getSerie("K2")
nextSerie=getSerie("K1")

nomsPropres=["Agathos","Clemencia","Elphaba","Freja","Hoderi","Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["agaTos","klemensja","elfaba","freZa","oderi","kaleb","maira","vjolet","nabil","katiSa","nikol"]

In [4]:
with open(prevSerie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.safe_load(stream)
with open(nextSerie+"Stems.yaml", 'r') as stream:
    nextStems=yaml.safe_load(stream)

In [5]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [6]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(nextStems)

In [7]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [8]:
with open(prevSerie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.safe_load(stream)

In [9]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [10]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical,choice="rdVowel"):
    result=list(radical)
    cibles=[]
    if not choice:
        choice=rd.choice(["apophonie","rdVowel"])
    if choice=="apophonie":
        for p,l in enumerate(radical):
            if l in "iae":
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=phonology["apophonies"][result[cible]]
            return "".join(result)
        else:
            return None
    else:
        for p,l in enumerate(radical):
            if l in voyelles:
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=rd.choice(voyelles)
            return "".join(result)
        else:
            return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [11]:
apophonie("kulubu",choice="rnd")

'kulubo'

In [12]:
operations=[
    devoiserFinale,
    apophonie,apophonie,
    voiserVCV,voiserVCV,
    metathese,
    changerLongueur
           ]

In [13]:
def modifierRadical(radical):
    nOperations=rd.choice([0,1,1,1,2,2,2,3,3,3,4,5])
    while nOperations>0:
        operation=rd.choice(operations)
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    return radical

In [14]:
racineChanges={}
prevVedettes={m.split(",")[0]:m for m in prevLexique}
nextVedettes={m.split(",")[0]:m for m in nextLexique}
print "lexique suivant"
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(prevLexique[mot])
#         print prevLexique[mot],racineChanges[nextLexique[mot]]
    else:
        print mot
        vedette=mot.split(",")[0]
        if vedette in nextVedettes:
            print "\t≠",nextVedettes[vedette]
        
print "================="
print "lexique précédent"
for mot in nextLexique:
    if not mot in prevLexique:
        print mot
        vedette=mot.split(",")[0]
        if vedette in prevVedettes:
            print "\t≠",prevVedettes[vedette]




lexique suivant
villageois.HYPER, villagEOIs, villagEOIS
enfant, enfants, garçon, garçons
	≠ enfant, enfants, enfANT, enfANTs
villageois, villageoiS
	≠ villageois, villageoiS, villagEOIs, villagEOIS
enfant.HYPER, enfANT, enfANTs
chasseur, chasseurs, chasseuse, chasseuses
	≠ chasseur, chasseurs
lexique précédent
chasseuse, chasseuses
garçon, garçons
chasseur, chasseurs
	≠ chasseur, chasseurs, chasseuse, chasseuses
villageois, villageoiS, villagEOIs, villagEOIS
	≠ villageois, villageoiS
enfant, enfants, enfANT, enfANTs
	≠ enfant, enfants, garçon, garçons


### Mots manquants

In [15]:
for i in range(10):
    print modifierRadical("Tonif")

Tonef
Tonof
Tonuf
Tonef
Tunafe
noDive
Tonof
Tonuf
niTif
naTif


In [16]:
racineChanges

{'DaDoS': 'nuguf',
 'Dapes': 'biZeb',
 'Dodum': 'senok',
 'DuNaN': 'gimil',
 'Duvel': 'duneN',
 'Nebun': 'siziS',
 'NiNut': 'Zekut',
 'NiZiN': 'NiruN',
 'Nidop': 'Zatid',
 'Nigud': 'TaZuT',
 'NoZub': 'famud',
 'Nonem': 'Zidip',
 'NuNip': 'fagij',
 'NuNuf': 'zanuT',
 'S': 'Si',
 'Serof': 'vabara',
 'SiToT': 'kuvave',
 'Siga': 'Tim',
 'Solid': 'tavoZo',
 'Sonap': 'sulif',
 'Sup': 'voZe',
 'Tabot': 'Zigon',
 'Tatik': 'nageS',
 'Tavat': 'Sodit',
 'Tebos': 'zurazi',
 'Temin': 'Sasib',
 'Tepap': 'godeD',
 'Tetit': 'gukepi',
 'TogoN': 'sovoti',
 'Tozop': 'Zeven',
 'TuNam': 'duduf',
 'TuZir': 'rikat',
 'Tutaw': 'lewibo',
 'Tuver': 'kader',
 'Tuzom': 'bazen',
 'Zabaju': 'tuk',
 'Zalep': 'fugog',
 'ZapaZ': 'NefTot',
 'Zaval': 'dolof',
 'ZeZeD': 'guZot',
 'ZiDaN': 'dobub',
 'Ziban': 'pigit',
 'Zirut': 'Nivudo',
 'ZoDol': 'sovog',
 'Zonar': 'kaZime',
 'agaTos': 'agaTos',
 'badoT': 'rikim',
 'bagud': 'Zipos',
 'balaz': 'ZagaT',
 'baloT': 'padis',
 'bame': 'fasip',
 'bazut': 'ZizaS',
 'beNar': 'dena

In [17]:
newStems=replace_keys(nextStems,racineChanges)

In [18]:
with open(nextSerie+'flowStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [19]:
with open(nextSerie+"flowStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [20]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  Donum: [blanc, blancs, blanche, blanches]
  Neval: [cinq]
  NigeS: [rouge, rouges]
  NiruN: [deux]
  Nuzer: [trois]
  ToSaS: [furieux, furieuse, furieuses]
  Tunas: [terrible, terribles]
  Zigon: [gros, grosse, grosses]
  denaZe: [vilain, vilains, vilaine, vilaines]
  gemabe: [différent, différents, différente, différentes]
  gimil: [dernier, derniers, dernière, dernières]
  gubuk: [vert, verts, verte, vertes]
  gumaz: [treize]
  kiDite: [bas, basse, basses]
  laroN: [petit, petits, petite, petites]
  liZan: [suivant, suivants, suivante, suivantes]
  lujef: [quatre]
  patoS: [courageux, courageuse, courageuses]
  poTiko: [nombreux, nombreuses]
  rapiZ: [méchant, méchants, méchante, méchantes]
  remes: [grand, grands, grande, grandes]
  rikim: [jaune, jaunes]
  rivuk: [effrayé, effrayés, effrayée, effrayées]
  rugego: [six]
  ruguk: [disparu, disparus, disparue, disparues]
  rurub: [blessé, blessés, blessée, blessées]
  siziS: [troisième, troisièmes]
  taTuk: [maigre, maigres]
  

In [21]:
with open(nextSerie+'newStems.yaml', 'w') as output:
    output.write(yamlText)

In [22]:
if 'VI' in nextStems['VER'].keys():
    prevVI=set()
    for cf,v in prevStems['VER']['VI'].iteritems():
        for k,l in v.iteritems():
            prevVI.add(l[0])
    nextVI=set()
    for cf,v in nextStems['VER']['VI'].iteritems():
        for k,l in v.iteritems():
            nextVI.add(l[0])

    print prevVI-nextVI
    print nextVI-prevVI
else:
    print "pas de distinction VI/VT"

pas de distinction VI/VT
